In [ ]:
import datetime
import time
import pandas as pd
from tqdm import tqdm
from google_play_scraper import Sort, reviews

In [ ]:
# Daftar ID aplikasi yang akan diambil ulasannya
app_ids = ['com.tokopedia.tkpd']

# Dictionary untuk menyimpan data ulasan per aplikasi
review_data_dict = {}

# Iterasi untuk setiap ID aplikasi
for app_id in tqdm(app_ids, desc="Memproses Aplikasi"):
    try:
        review_data = []  # List untuk menyimpan data ulasan yang sesuai tanggal
        continuation_token = None  # Token untuk pagination
        matching_review_count = 0  # Jumlah ulasan yang sesuai dengan rentang tanggal
        non_matching_review_count = 0  # Jumlah ulasan di luar rentang tanggal

        # Rentang tanggal yang diinginkan (1 Januari 2025 hingga 27 Mei 2025)
        end_date = datetime.datetime(2025, 5, 27, 23, 59, 59)
        start_date = datetime.datetime(2025, 1, 1, 0, 0, 0)

        # Inisialisasi progress bar untuk ulasan
        review_bar = tqdm(desc=f'Ulasan untuk {app_id}: ', total=500)

        # Loop untuk mengambil semua ulasan sesuai rentang tanggal
        while True:
            review_result, continuation_token = reviews(
                app_id,
                lang='id',              # Bahasa Indonesia
                country='id',           # Negara Indonesia
                sort=Sort.NEWEST,       # Urutkan dari yang terbaru
                count=100,              # Ambil 100 ulasan per batch
                continuation_token=continuation_token
            )

            # Filter ulasan sesuai rentang tanggal
            filtered_reviews = [review for review in review_result if start_date <= review['at'] <= end_date]
            review_data.extend(filtered_reviews)

            # Hitung jumlah ulasan yang cocok dan tidak cocok dengan rentang waktu
            for review in review_result:
                if review['at'] < start_date:
                    non_matching_review_count += 1
                elif start_date <= review['at'] <= end_date:
                    matching_review_count += 1
                    non_matching_review_count = 0
                else:
                    non_matching_review_count = 0

            # Update progress bar
            review_bar.update(1)
            review_bar.set_postfix({
                'Cocok': matching_review_count,
                'Tidak Cocok': non_matching_review_count
            })

            # Hentikan jika tidak ada token berikutnya atau terlalu banyak ulasan di luar rentang
            if not continuation_token or non_matching_review_count >= 500:
                break

        review_bar.close()

        # Simpan data hasil scraping ke DataFrame
        kolom_data = ['reviewId', 'userName', 'content', 'score', 'thumbsUpCount', 'at', 'reviewCreatedVersion', 'appVersion']
        review_data_dict[app_id] = pd.DataFrame(review_data, columns=kolom_data)

        # Tunggu 5 detik sebelum lanjut ke aplikasi berikutnya (untuk menghindari pembatasan)
        time.sleep(5)

    except Exception as e:
        print(f'Error saat mengambil ulasan untuk aplikasi {app_id}: {e}')

  0%|          | 0/1 [00:00<?, ?it/s]

Review count for com.tokopedia.tkpd:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
df = review_data_dict['com.tokopedia.tkpd'].head()
print(df.shape)
df.head()

(17881, 9)


,reviewId,userName,content,score,thumbsUpCount,at,reviewCreatedVersion,appVersion,clean_content
0,d92ebb06-3dc9-4ce8-98e0-b4ccf565dc67,UMS HERBAL INDONESIA,Jangan keseringan update dong... Bikin kesel t...,2,0,2025-05-27 02:38:46,3.313.0,3.313.0,jangan keseringan update dong bikin kesel tiap...
1,20a3af68-5e34-4abf-be01-db5f4d042cd5,Siti Sukaesih,Pelayanan dan respon oke mantap,5,0,2025-05-27 02:38:21,3.303.0,3.303.0,pelayanan dan respon oke mantap
2,1f2276f1-b51e-498a-b56c-7f4a5ed1ad04,Dedi Hay,lagi di coba dulu,4,0,2025-05-27 02:34:55,3.313.0,3.313.0,lagi di coba dulu
3,776974d8-3f01-4531-abf2-7cf5ce2b4172,Wahyu Saputra,bagus,5,0,2025-05-27 02:20:59,3.313.0,3.313.0,bagus
4,f534201c-2a5e-4f89-a2ab-5249f4f0131c,Hamda Sakhiyah,saya belanja disini selalu dpt harga murah,5,0,2025-05-27 02:11:54,3.313.0,3.313.0,saya belanja disini selalu dpt harga murah


In [ ]:
df.to_csv("../datasets/app_review_data_com.tokopedia.tkpd.csv", index=False)